In [1]:
import numpy as np 

def GE( U , b ) : 
    '''
    Param U: nxn matrix of the coefficients. ( Upper Trangular ) 
    Param b: nx1 matrix of the constants in the equation. 
    '''
    if( type( U ) == list ):
        U = np.array( U , dtype = float  ) 
    if( type( b ) == list ) :
        b = np.array( b , dtype = float )

    n = U.shape[0] 
    
    if( n != U.shape[1] ) : 
        raise ValueError('U must be a square matrix') 
    if( n != b.shape[0] ) : 
        raise ValueError('Dimentions of U and b do not match' ) 
    
    b = b.reshape( ( n, -1 ) ) 
    U = np.concatenate( ( U , b ) , axis = 1  )

    for i in range( n - 1 ) : 
        for j in range( i + 1 , n ) : 
            U[j,:] -= ( U[j,i]/U[i,i] )*U[i,:] 

    return U[: , :n ] , U[: , -1 ] 


In [2]:
# back substitution 

def BackSt( U , b ): 
    '''
    Param U: nxn matrix of the coefficients. ( Upper Trangular ) 
    Param b: nx1 matrix of the constants in the equation. 
    '''
    if( type( U ) == list ) :
        U = np.array( U , dtype = float  ) 
    if( type( b ) == list ) :
        b = np.array( b , dtype = float )

    n = U.shape[0] 
    
    if( n != U.shape[1] ) : 
        raise ValueError('U must be a square matrix') 
    if( n != b.shape[0] ) : 
        raise ValueError('Dimentions of U and b do not match' ) 

    x = np.zeros( n )
    
    x[n-1] = b[n-1]/U[ n -1 , n -1 ] 

    for i in range( n -2 , -1 , -1 ) : 
        temp = .0 
        for j in range( i + 1 , n ) : 
            temp += x[j]*U[i,j]
        x[i] = ( b[i] - temp )/U[i,i] 
    
    return x

In [3]:
f1 = lambda x , y : x*y + x**2 - y**3 -1  
f2 = lambda x , y : x + 2*y - x*(y**2) - 2 

f1x = lambda x,y: y + 2*x 
f1y = lambda x,y: x -3*(y**2) 
f2x = lambda x,y: 1 - y**2 
f2y = lambda x,y: 2 - 2*x*y

def get_jacobian(x1,x2):
    return np.array([
        [f1x(x1,x2),f1y(x1,x2)],
        [f2x(x1,x2),f2y(x1,x2)]
    ])  

def f(x1,x2): 
    return np.array(
        [
            [f1(x1,x2)], 
            [f2(x1,x2)]
        ]
    )
    
def to_vector(x1,x2): 
    return np.array([
        [x1],
        [x2]
    ])
    
def get_euclidean(x1,x2): 
    return np.sqrt( x1**2 + x2**2 )

In [12]:
def NRM(x1i,x2i,max_itr=100,tor=1e-4): 
    err , x1,x2 = [None] , [x1i] , [x2i]
    fval = [f(x1i,x2i)]
    itr = 1
    while (err[-1] == None or err[-1] > tor ) and itr < max_itr : 
        U , b = GE( get_jacobian(x1[-1],x2[-1]) , fval[-1] )
        dx = BackSt(U,b)
        dx1 , dx2 = dx 
        x1.append( x1[-1] - dx1 ) 
        x2.append(  x2[-1] - dx2 )
        err.append( get_euclidean(dx1,dx2)/(get_euclidean(x1[-1],x2[-1])+1e-7) )
        fval.append(f(x1[-1],x2[-1]))
        itr += 1 
    return x1, x2  , fval , err , itr 

In [13]:
x1,x2 , fval , err , itr = NRM(1.,2.)
x1[-1] , x2[-1] ,  fval[-1] , err[-1] , itr 

(1.000050948403376,
 1.000076422605475,
 array([[-1.10331633e-08],
        [-1.36279314e-08]]),
 6.493974903851329e-05,
 14)